coding=utf-8
Copyright 2024 The Google Research Authors.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

"""X-MAGICAL cross-embodiment pretraining script."""


In [ ]:

import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


In [ ]:

# Mapping from pretraining algorithm to config file.
ALGO_TO_CONFIG = {
    "xirl": "configs/xmagical/pretraining/tcc.py",
    "lifs": "configs/xmagical/pretraining/lifs.py",
    "tcn": "configs/xmagical/pretraining/tcn.py",
    "goal_classifier": "configs/xmagical/pretraining/classifier.py",
    "raw_imagenet": "configs/xmagical/pretraining/imagenet.py",
}


In [ ]:
# We want to pretrain on the entire 1k demonstrations.
MAX_DEMONSTRATIONS = -1
# embodiment = None
embodiment = "longstick"
algo = "xirl"
unique_name = True,
random_number = random.randint(1, 1000)  # You can adjust the range as needed
experiment_name = f"/home/user/xirl/exp/exp{random_number}"
experiment_name = "/home/user/xirl/exp/same_long_gnn"
dataset_folder = f'/tmp/xirl/datasets/xmagical/train/{embodiment}'
# device = 'cpu'
device = 'cuda:0'
embodiments = EMBODIMENTS if embodiment is None else [embodiment]
graph_data_path = f"./data/{embodiment}/graphs/combined_graph.pt"
print("experiment name: ",experiment_name)

In [ ]:
for embodiment in embodiments:
    # Generate a unique experiment name.
    print("embodiment: ",embodiment)
    kwargs = {
        "dataset": "xmagical",
        "mode": "cross",
        "algo": algo,
        "embodiment": embodiment,
    }
    if unique_name:
      kwargs["uid"] = unique_id()
    logging.info("Experiment name: %s", experiment_name)
    
    # Train on all classes but the given embodiment.
    trainable_embs = tuple(EMBODIMENTS - set([embodiment]))

In [ ]:
for embodiment in embodiments:
    # The 'goal_classifier' baseline does not need to compute a goal embedding.
      subprocess.run(
          [
              "python",
              "compute_goal_embedding_gnn.py",
              "--experiment_path",
              experiment_name,
              "--graph_data_path",
              graph_data_path,
              "--dataset_folder",
              dataset_folder
          ],
          check=True,
      )


In [ ]:
import utils

gnn_emb = utils.load_pickle(experiment_name, "gnn_emb.pkl")
gnn_distance_scale = utils.load_pickle(experiment_name, "gnn_distance_scale.pkl")

print(gnn_emb,gnn_distance_scale)
